# Задача D: Проверка ацикличности графа

Источник: [лаба про DFS](http://cs.mipt.ru/algo/lessons/sem_2/08.graph_dfs.html).

Дан ориентированный граф.
Вершины пронумерованы от 0.
Трeбуется с помощью обхода в глубину проверить, является ли граф ацикличным.

## Формат входных данных

На вход программе в первой строке подаются через пробел два числа: N (2 <= N <= 1000) — число вершин в графе и M (1 <= M <= 20000) — число рёбер.
В следующих M строках задаются рёбра, по два числа в каждой строке — номера соединённых вершин (соответствующее ребро идет из первой вершины во вторую).

## Формат выходных данных

Требуется распечатать номера вершин, задающих цикл в графе, если он существует.
Номера вершин нужно вывести в порядке следования по циклу.
Если циклов несколько, вывести любой.
Если граф ацикличный, вывести строку "YES" без кавычек.

## Примеры

Пример 1:
```
Вход:

4 1
3 1

Выход:

YES
```

Пример 2:
```
Вход:

8 8
7 3
3 1
5 2
6 4
6 7
4 7
1 6
1 2

Выход:

1 6 7 3
```

## Решение

### Считывание данных ("сырой" граф)

In [1]:
V, E = [int(n) for n in input().split()]

edges = []

for _ in range(E):
    v1, v2 = [int(n) for n in input().split()]
    edges.append((v1, v2))

8 8
7 3
3 1
5 2
6 4
6 7
4 7
1 6
1 2


### "Причёсывание" графа

с прицелом на последующий DFS для поиска цикла

In [2]:
class Node:
    def __init__(self, value):
        self.value = value
        self.descendants = list()
        
        # Предыдущая вершина в пути, который получается при обходе в глубину
        # (может меняться в процессе: когда при обходе спускаемся,
        #  когда поднимаемся, или когда приходим из другой вершины)
        self.previous = False
        
        # Посещена — сразу когда первый раз заходим в вершину при обходе графа в глубину
        self.is_visited = False
        
        # Проводили ли поиск в глубину из данной вершины
        self.is_dfsed = False
    
    def __eq__(self, other):
        return self.value == other.value

In [3]:
nodes = [
    Node(i) for i in range(V)
]

for edge in edges:
    v1, v2 = edge
    nodes[v1].descendants.append(nodes[v2])  # пользуемся тем, что вершины пронумерованы

### Поиск цикла

In [4]:
def find_cycle_dfs(root):
    if root.is_dfsed:
        return None
    
    root.is_visited = True
    
    for d in root.descendants:
        if d.is_dfsed:    # не цикл
            continue
        
        if d.is_visited:  # цикл
            return restore_cycle(first=d, last=root)
        
        d.previous = root
        cycle = find_cycle_dfs(d)
        
        if cycle is not None:
            return cycle
    
    root.is_dfsed = True
    
    return None


def restore_cycle(first, last):
    reversed_cycle = [last]

    while (previous := reversed_cycle[-1].previous) != first:  # "Моржовый" оператор
                                                               # https://realpython.com/python-walrus-operator
        reversed_cycle.append(previous)

    reversed_cycle.append(first)
    cycle = reversed_cycle[::-1]

    return cycle

In [5]:
for node in nodes:
    if node.is_dfsed:
        continue
    
    cycle = find_cycle_dfs(node)
    
    if cycle is not None:
        cycle_values = [n.value for n in cycle]
        
        print(*cycle_values)
        
        break
else:
    print('YES')

1 6 4 7 3


### P.S.

Про `else` после `for` или `while` цикла: https://www.geeksforgeeks.org/using-else-conditional-statement-with-for-loop-in-python.

(TL;DR: вообще полезно, но имя `else` [не самое удачное](https://stackoverflow.com/questions/9979970/why-does-python-use-else-after-for-and-while-loops#comment12751930_9979970)).